# 1. Setup

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Run a local elasticsearch
https://www.elastic.co/guide/en/elasticsearch/reference/current/docker.html#docker-cli-run-dev-mode

```bash
docker run -d -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.4.1
```

In [10]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200/'); res.json()

{'name': '7bb755addb91',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'gVT_966-TZiYuOQSHPPxYg',
 'version': {'number': '7.4.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'fc0eeb6e2c25915d63d871d344e3d0b45ea0ea1e',
  'build_date': '2019-10-22T17:16:35.176724Z',
  'build_snapshot': False,
  'lucene_version': '8.2.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### Install dependencies

In [11]:
!pip install elasticsearch pandas

In [12]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
pd.set_option('display.max_colwidth', 500) 
pd.set_option('display.max_columns', None)

### Create elastic client

In [13]:
es_client = Elasticsearch('http://localhost:9200')

# 2. Get and load some data

### Load dataset

In [14]:
url = "https://raw.githubusercontent.com/algolia/datasets/master/ecommerce/records.json"
df = pd.read_json(url).set_index('objectID').rename(columns={"name": "product_name"}); len(df)

10000

In [15]:
df.head(2)

,brand,categories,description,free_shipping,hierarchicalCategories,image,product_name,popularity,price,price_range,rating,type,url
objectID,,,,,,,,,,,,,
1696302,Pogoplug,"[Best Buy Gift Cards, Entertainment Gift Cards]","Enjoy 3 years of unlimited Cloud storage service with this activation card, which allows you to remotely access your favorite music, movies and other media via a compatible device and enables private file sharing with loved ones.",True,"{'lvl0': 'Best Buy Gift Cards', 'lvl1': 'Best Buy Gift Cards > Entertainment Gift Cards'}",https://cdn-demo.algolia.com/bestbuy/1696302_sc.jpg,3-Year Unlimited Cloud Storage Service Activation Card - Other,10000,69.00,50 - 100,2,Online data backup,http://www.bestbuy.com/site/3-year-unlimited-cloud-storage-service-activation-card-other/1696302.p?id=1219066776306&skuId=1696302&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W
9131042,360fly,"[Cameras & Camcorders, Digital Cameras, Point & Shoot Cameras, 360 & Panoramic Cameras]","Only at Best Buy This 360fly panoramic 360&#176; HD video camera features Wi-Fi, a Bluetooth LE interface and the included 360fly app (download required), so you can easily shoot, edit, replay and share recorded video footage using a compatible iOS or Android smartphone. Included action camera adaptor lets you use your 360fly camera with your existing 3-prong mounts.",False,"{'lvl0': 'Cameras & Camcorders', 'lvl1': 'Cameras & Camcorders > Digital Cameras', 'lvl2': 'Cameras & Camcorders > Digital Cameras > Point & Shoot Cameras', 'lvl3': 'Cameras & Camcorders > Digital Cameras > Point & Shoot Cameras > 360 & Panoramic Cameras'}",https://cdn-demo.algolia.com/bestbuy/9131042_rb.jpg,360fly - Panoramic 360° HD Video Camera - Black,9999,399.99,200 - 500,5,Point&#38;shoot camcrder,http://www.bestbuy.com/site/360fly-panoramic-360-hd-video-camera-black/9131042.p?id=1219405459683&skuId=9131042&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W


### Load Data to Elasticsearch

**Elasticsarch Documents:**

* _index:    Indicating to which index the documents belong 
* _id:       Unique ID of your document (useful to easily update / remove)
* _source:   Contains the fields of your document

In [16]:
def doc_generator(df, es_index):
    return ({
            "_index": es_index,
            "_id": idx,
            "_source": es_record.to_dict(),
    } for idx, es_record in df.iterrows())

Loading data in bulk to Elasticsearch

In [17]:
def load_data(es_client, es_index, df):
    return helpers.bulk(
        client=es_client,
        actions=doc_generator(df, es_index=es_index)
    )

In [18]:
load_data(es_client, 'test', df)

(10000, [])

# 2. Query Elasticsearch

Many different query are provided in Elasticsearch. Let's start by retrieving a single document:

In [19]:
# Query 1 of the documents
import json

print(json.dumps(
    es_client.get(index='test', id='1696302'),
    indent=2)
 )

{
  "_index": "test",
  "_type": "_doc",
  "_id": "1696302",
  "_version": 1,
  "_seq_no": 0,
  "_primary_term": 1,
  "found": true,
  "_source": {
    "brand": "Pogoplug",
    "categories": [
      "Best Buy Gift Cards",
      "Entertainment Gift Cards"
    ],
    "description": "Enjoy 3 years of unlimited Cloud storage service with this activation card, which allows you to remotely access your favorite music, movies and other media via a compatible device and enables private file sharing with loved ones.",
    "free_shipping": true,
    "hierarchicalCategories": {
      "lvl0": "Best Buy Gift Cards",
      "lvl1": "Best Buy Gift Cards > Entertainment Gift Cards"
    },
    "image": "https://cdn-demo.algolia.com/bestbuy/1696302_sc.jpg",
    "product_name": "3-Year Unlimited Cloud Storage Service Activation Card - Other",
    "popularity": 10000,
    "price": 69.0,
    "price_range": "50 - 100",
    "rating": 2,
    "type": "Online data backup",
    "url": "http://www.bestbuy.com/site/

### The "query_string"

Using the search API of Elasticsearch (https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html)

In [20]:
def query_elasticsearch(index, query):
    return es_client.search(
        index=index,
        body=query
    )

Returns documents based on a provided query string

In [21]:
def basic_query_string(search_terms):
    return {
        "query": {
            "query_string" : {
                "query" : search_terms
            }
        }
    }

Let's build a small function to display nicely the JSON obtain in response

In [22]:
from pandas.io.json import json_normalize

def display_results(resp):
    return json_normalize(resp['hits']['hits'])

Let's try a simple query

In [23]:
resp = query_elasticsearch("test", basic_query_string("car audio"))
display_results(resp)[:10]

,_id,_index,_score,_source.brand,_source.categories,_source.description,_source.free_shipping,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.hierarchicalCategories.lvl3,_source.hierarchicalCategories.lvl4,_source.image,_source.popularity,_source.price,_source.price_range,_source.product_name,_source.rating,_source.type,_source.url,_type
0,3500126,test,12.631571,Kenwood,"[Car Electronics & GPS, Car Audio, Car Stereo Receivers, In-Dash with HD Radio, Built-In HD Radio Decks]","Enjoy your favorite music with this Kenwood KDC-BT762HD in-dash deck, which features CD and MP3 playback, plus Apple&#174; iPhone&#174; and iPod&#174; and Android connectivity for versatile playback options. Built-in Bluetooth enables wireless streaming.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Stereo Receivers,Car Electronics & GPS > Car Audio > Car Stereo Receivers > In-Dash with HD Radio,Car Electronics & GPS > Car Audio > Car Stereo Receivers > In-Dash with HD Radio > Built-In HD Radio Decks,https://cdn-demo.algolia.com/bestbuy/3500126_sb.jpg,4712,169.99,100 - 200,Kenwood - CD - Built-in Bluetooth - Built-In HD Radio - In-Dash Deck with Detachable Faceplate - Variable Color,3,Cd decks,http://www.bestbuy.com/site/kenwood-cd-built-in-bluetooth-built-in-hd-radio-in-dash-deck-with-detachable-faceplate-variable-color/3500126.p?id=1219582881375&skuId=3500126&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
1,9225581,test,9.707445,Griffin Technology,"[Audio, Cables & Chargers, MP3 Player Cables]",Easily connect a portable audio device to your home or car stereo system with this convenient coiled cable.,False,Audio,Audio > Cables & Chargers,Audio > Cables & Chargers > MP3 Player Cables,NaN,NaN,https://cdn-demo.algolia.com/bestbuy/9225581_sb.jpg,6413,22.99,1 - 50,Griffin Technology - StereoConnect Cable for Most Portable Audio Devices,5,Connectivity,http://www.bestbuy.com/site/griffin-technology-stereoconnect-cable-for-most-portable-audio-devices/9225581.p?id=1218062421800&skuId=9225581&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
2,8063027,test,9.416204,Pioneer,"[Car Electronics & GPS, Car Audio, Car Speakers, 5.25"" Car Speakers]","Enjoy your favorite music in your car with these Pioneer TS-G1345R 2-way car speakers, which feature 250W maximum power handling to deliver sounds in robust detail. The rubber-coated cloth surrounds help ensure high-fidelity audio.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 5.25"" Car Speakers",NaN,https://cdn-demo.algolia.com/bestbuy/8063027_sb.jpg,2101,49.99,1 - 50,"Pioneer - 5-1/4"" 2-Way Car Speakers with IMPP Composite Cones (Pair) - Black",2,Speakers,http://www.bestbuy.com/site/pioneer-5-1-4-2-way-car-speakers-with-impp-composite-cones-pair-black/8063027.p?id=1219309055448&skuId=8063027&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
3,7456254,test,9.194889,Pioneer,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]","Pioneer TS-A6965R Car Speakers: If you love listening to music in your car, it's time to upgrade from your factory speakers. These Pioneer speakers deliver the in-car audio experience you want for real driving enjoyment. Geek Squad&#174; installation is strongly recommended for this product. Please visit your local Best Buy store for details.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",NaN,https://cdn-demo.algolia.com/bestbuy/7456254_sb.jpg,2095,79.99,50 - 100,"Pioneer - 6"" x 9"" 3-Way Car Speakers with Multilayer Mica Matrix Cones (Pair) - Black",2,Speakers,http://www.bestbuy.com/site/pioneer-6-x-9-3-way-car-speakers-with-multilayer-mica-matrix-cones-pair-black/7456254.p?id=1218846426021&skuI

The results are showing some audio cables that have little to do with a car ...

### Field Boosting

We can boost specific field in our query (https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-boost.html)

In [24]:
# Give higher weight to categories
def query_with_weight(search_terms):
    return {
        "query": {
            "query_string" : {
                "query" : search_terms,
                "fields": [
                    "description",
                    "hierarchicalCategories.lvl0^2",
                    "hierarchicalCategories.lvl1^2",
                    "hierarchicalCategories.lvl2^2",
                    "hierarchicalCategories.lvl3^2",
                    "product_name",
                    "type"
                ]
            }
        }
    }

resp = query_elasticsearch("test", query_with_weight("car audio"))
display_results(resp)[:10]

,_id,_index,_score,_source.brand,_source.categories,_source.description,_source.free_shipping,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.hierarchicalCategories.lvl3,_source.image,_source.popularity,_source.price,_source.price_range,_source.product_name,_source.rating,_source.type,_source.url,_type
0,1807279,test,13.898865,Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 6.5"" Car Speakers]","These speakers feature 6-1/2"" poly-mica woofer cones and 1"" silk swivel soft-dome tweeters that deliver powerful, smooth sound. The high-strength motor structure and high-excursion design provide accurate bass performance.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6.5"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/1807279_sb.jpg,9911,99.99,50 - 100,"Alpine - 6-1/2"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black",1,Speakers,http://www.bestbuy.com/site/alpine-6-1-2-2-way-coaxial-car-speakers-with-poly-mica-cones-pair-black/1807279.p?id=1218297519264&skuId=1807279&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
1,1416147,test,13.898865,Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 6.5"" Car Speakers]","Listen to your favorite tunes on the go with these 2-way coaxial speakers that feature 240 watts of peak power for impressive sound quality and a 3/4"" silk soft-dome tweeter for clean audio.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6.5"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/1416147_sb.jpg,9910,79.99,50 - 100,"Alpine - 6-1/2"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black",3,Speakers,http://www.bestbuy.com/site/alpine-6-1-2-2-way-coaxial-car-speakers-with-polypropylene-cones-pair-black/1416147.p?id=1218256568420&skuId=1416147&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
2,1807251,test,13.898865,Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 6.5"" Car Speakers]","Listen to your favorite songs in your car with these 2-way speakers that feature 70Hz - 22kHz frequency response for accurate sound reproduction. The poly-mica woofer cones and 1"" silk tweeters deliver clear audio.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6.5"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/1807251_sb.jpg,9909,129.99,100 - 200,"Alpine - 6-1/2"" 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black",1,Speakers,http://www.bestbuy.com/site/alpine-6-1-2-2-way-component-car-speakers-with-poly-mica-cones-pair-black/1807251.p?id=1218293491912&skuId=1807251&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
3,2305807,test,13.898865,Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 5.25"" Car Speakers]",Listen to your favorite music in your car with these speakers that feature a neodymium magnet and hybrid fiber cones for powerful sound.,False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 5.25"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/2305807_sb.jpg,9900,199.99,100 - 200,"Alpine - Type R 5-1/4"" 2-Way Coaxial Car Speakers with Hybrid Fiber Cones (Pair) - Black",3,Speakers,http://www.bestbuy.com/site/alpine-type-r-5-1-4-2-way-coaxial-car-speakers-with-hybrid-fiber-cones-pair-black/2305807.p?id=1218320406807&skuId=2305807&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
4,7551355,test,13.898865,Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 5.25"" Car Speakers]","Upgrade your car audio system with these Alpine Type-S SPS-510 2-way coaxial car speakers, which handle up to 170W peak power and feature 5-1

Better already. But what if I am looking for some specific size of speaker?

In [25]:
resp = query_elasticsearch("test", query_with_weight("car audio 6 inches"))
display_results(resp)[:10]

,_id,_index,_score,_source.brand,_source.categories,_source.description,_source.free_shipping,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.hierarchicalCategories.lvl3,_source.image,_source.popularity,_source.price,_source.price_range,_source.product_name,_source.rating,_source.type,_source.url,_type
0,1807206,test,21.872923,Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]","Enjoy your favorite songs on the go with these coaxial 2-way car speakers that feature 65Hz - 18kHz frequency response for accurate sound reproduction. The 3/4"" silk soft-dome tweeter provides crisp, unhindered sound.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/1807206_sb.jpg,9912,89.99,50 - 100,"Alpine - 6"" x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black",4,Speakers,http://www.bestbuy.com/site/alpine-6-x-9-2-way-coaxial-car-speakers-with-polypropylene-cones-pair-black/1807206.p?id=1218293494853&skuId=1807206&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
1,7979429,test,21.872923,Infinity,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]","Improve your vehicle's audio system with these Infinity Primus PR9603IS 3-way car speakers that feature a 53Hz - 21kHz frequency response to reproduce sound accurately. The 6"" x 9"" Plus One polypropylene woofer cones produce high-quality sound.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/7979429_sb.jpg,5757,99.99,50 - 100,"Infinity - Primus 6"" x 9"" 3-Way Car Speakers with Polypropylene Cones (Pair) - Black",3,Speakers,http://www.bestbuy.com/site/infinity-primus-6-x-9-3-way-car-speakers-with-polypropylene-cones-pair-black/7979429.p?id=1218863183338&skuId=7979429&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
2,8038064,test,21.872923,JBL,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 8"" Car Speakers]","These JBL GX862 2-way coaxial car speakers feature 6"" x 8"" polypropylene woofer cones and 3/4"" edge-driven, soft-dome tweeters for crisp, accurate reproduction of your favorite music. The speakers can be installed in a 5"" x 7"" cutout for versatility.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 8"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/8038064_sb.jpg,4959,119.99,100 - 200,"JBL - 6"" x 8"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black",1,Speakers,http://www.bestbuy.com/site/jbl-6-x-8-2-way-coaxial-car-speakers-with-polypropylene-cones-pair-black/8038064.p?id=1219307883865&skuId=8038064&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
3,8038055,test,21.872923,JBL,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]","With a 45Hz - 21kHz frequency response, these JBL GX963 3-way car speakers utilize 6"" x 9"" polypropylene woofer cones to deliver a complete range of lush, high-fidelity tones throughout your vehicle as you drive.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/8038055_sb.jpg,4958,129.99,100 - 200,"JBL - 6"" x 9"" 3-Way Car Speakers with Polypropylene Cones (Pair) - Black",1,Speakers,http://www.bestbuy.com/site/jbl-6-x-9-3-way-car-speakers-with-polypropylene-cones-pair-black/8038055.p?id=1219307886491&skuId=8038055&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
4,3936018,test,21.872923,Kenwood,"[Car Electronics & GPS, Car Audio, 

Unfortunately that query does not work very well :/  Let's see what we can do to fix that

# 3. Mapping (Schema)

In [26]:
def create_elasticsearch_mapping(index, mapping):
    # ignore 400 cause by IndexAlreadyExistsException when creating an index
    return es_client.indices.create(index=index, body=mapping, ignore=400)

def delete_elasticsearch_mapping(index):
    return es_client.indices.delete(index=index, ignore=[400, 404])

Each field has a data type which can be:

* a simple type like text, keyword, date, long, double, boolean or ip.
* a type which supports the hierarchical nature of JSON such as object or nested.
* mor a specialised type like geo_point, geo_shape, or completion.

In [27]:
es_mapping = {
    "settings" : {
       "number_of_shards": 5,
        "number_of_replicas": 1
    },
    "mappings": {
        "properties": {
            "product_name":           { "type": "text" },  
            "description":            { "type": "text" },
            "hierarchicalCategories": {
                "type": "nested",
                "include_in_parent": "true",
                "properties": {
                    "lvl0": {"type": "text"},
                    "lvl1": {"type": "text"},
                    "lvl2": {"type": "text"},
                    "lvl3": {"type": "text"},
                    "lvl4": {"type": "text"}
                }
            },
            "type":                   { "type": "text" },
            "url":                    { "type": "text", "index" : False },
            "image":                  { "type": "text", "index" : False },
            "popularity":             { "type": "double" },
            "price":                  { "type": "text" },
            "price_range":            { "type": "keyword" },
            "rating":                 { "type": "short" },
            "free_shipping":          { "type": "boolean" }
    }
  }
}

delete_elasticsearch_mapping('test-w-mapping')
create_elasticsearch_mapping("test-w-mapping", es_mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test-w-mapping'}

In [28]:
load_data(es_client, 'test-w-mapping', df)

(10000, [])

In [29]:
resp = query_elasticsearch("test-w-mapping", query_with_weight("car audio 6 inches"))
display_results(resp)[:10]

,_id,_index,_score,_source.brand,_source.categories,_source.description,_source.free_shipping,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.hierarchicalCategories.lvl3,_source.image,_source.popularity,_source.price,_source.price_range,_source.product_name,_source.rating,_source.type,_source.url,_type
0,1807206,test-w-mapping,22.940664,Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]","Enjoy your favorite songs on the go with these coaxial 2-way car speakers that feature 65Hz - 18kHz frequency response for accurate sound reproduction. The 3/4"" silk soft-dome tweeter provides crisp, unhindered sound.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/1807206_sb.jpg,9912,89.99,50 - 100,"Alpine - 6"" x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black",4,Speakers,http://www.bestbuy.com/site/alpine-6-x-9-2-way-coaxial-car-speakers-with-polypropylene-cones-pair-black/1807206.p?id=1218293494853&skuId=1807206&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
1,6432002,test-w-mapping,22.649021,Kicker,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]",Blast your favorite music while cruising with these Kicker KS Series 41KSC6934 3-way speakers that feature EVC (Extended Voice Coil) technology for thumping bass response. The calibrated-dome tweeters help you hear every detail.,False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/6432002_sb.jpg,4602,149.99,100 - 200,"Kicker - KS Series 6"" x 9"" 3-Way Car Speakers with Polymer Cones (Pair) - Black",3,Speakers,http://www.bestbuy.com/site/kicker-ks-series-6-x-9-3-way-car-speakers-with-polymer-cones-pair-black/6432002.p?id=1219217424553&skuId=6432002&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
2,8063256,test-w-mapping,22.649021,Pioneer,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 8"" Car Speakers]","Featuring 6"" x 8"" IMPP composite woofer cones and 1-3/16"" PET Piezoelectric tweeters, these Pioneer TS-G6845R car speakers project a wide range of crisp, robust tones, so you can enjoy listening to music while you drive.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 8"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/8063256_sb.jpg,2098,49.99,1 - 50,"Pioneer - 6"" x 8"" 2-Way Car Speakers with IMPP Composite Cones (Pair) - Black",1,Speakers,http://www.bestbuy.com/site/pioneer-6-x-8-2-way-car-speakers-with-impp-composite-cones-pair-black/8063256.p?id=1219309057333&skuId=8063256&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
3,3937008,test-w-mapping,22.085585,Kenwood,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]","These Kenwood Performance Series KFC-6995PS car speakers feature 6"" x 9"" paper woofer cones, 1/2"" PET-dome and 3/8"" ceramic super tweeters and 13/16"" PET balanced-dome tweeters for vivid sonic output. The acoustic sound harmonizer offers clear audio.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/3937008_rb.jpg,4707,99.99,50 - 100,"Kenwood - Performance Series 6"" x 9"" 5-Way Car Speakers with Paper Cones (Pair) - Black",6,Speakers,http://www.bestbuy.com/site/kenwood-performance-series-6-x-9-5-way-car-speakers-with-paper-cones-pair-black/3937008.p?id=1219094699970&skuId=3937008&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
4,7456272,test-w-mapping

Still not working ...

# 4. Analyzers

<img src="https://www.elastic.co/guide/en/elasticsearch/client/net-api/current/analysis-chain.png" alt="analyzers in elasticsearch" width="600"/>

One possible solution is to convert the char during analysis. Let's add the `char_filter` to the default `standard` analyzer (https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-standard-analyzer.html)

In [30]:
es_mapping_v2 = {
    "settings" : {
        "number_of_shards": 5,
        "number_of_replicas": 1,
        "analysis": {
            "analyzer": {
                "my_analyzer": {
                    "tokenizer": "standard",
                    "char_filter": [
                        "my_char_filter"
                    ]
                }
            },
            "char_filter": {
                "my_char_filter": {
                    "type": "pattern_replace",
                    "pattern": "([0-9]+.?[0-9]?)\"",
                    "replacement": "$1 inch"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "product_name":           { "type": "text" },  
            "description":            { "type": "text" },
            "hierarchicalCategories": {
                "type": "nested",
                "include_in_parent": "true",
                "properties": {
                    "lvl0": {"type": "text"},
                    "lvl1": {"type": "text"},
                    "lvl2": {"type": "text"},
                    "lvl3": {"type": "text", "analyzer": "my_analyzer"},
                    "lvl4": {"type": "text"}
                }
            },
            "type":                   { "type": "text" },
            "url":                    { "type": "text", "index" : False },
            "image":                  { "type": "text", "index" : False },
            "popularity":             { "type": "double" },
            "price":                  { "type": "text" },
            "price_range":            { "type": "keyword" },
            "rating":                 { "type": "short" },
            "free_shipping":          { "type": "boolean" }
    }
  }
}

delete_elasticsearch_mapping('test-w-mapping-v2')
create_elasticsearch_mapping("test-w-mapping-v2", es_mapping)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'test-w-mapping-v2'}

Let's load some data in our new index

In [31]:
load_data(es_client, 'test-w-mapping-v2', df)

(10000, [])

Let's see if that works!

In [32]:
resp = query_elasticsearch('test-w-mapping-v2', query_with_weight("car audio 6 inches"))
display_results(resp)[:10]

,_id,_index,_score,_source.brand,_source.categories,_source.description,_source.free_shipping,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.hierarchicalCategories.lvl3,_source.image,_source.popularity,_source.price,_source.price_range,_source.product_name,_source.rating,_source.type,_source.url,_type
0,1807206,test-w-mapping-v2,23.699783,Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]","Enjoy your favorite songs on the go with these coaxial 2-way car speakers that feature 65Hz - 18kHz frequency response for accurate sound reproduction. The 3/4"" silk soft-dome tweeter provides crisp, unhindered sound.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/1807206_sb.jpg,9912,89.99,50 - 100,"Alpine - 6"" x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black",4,Speakers,http://www.bestbuy.com/site/alpine-6-x-9-2-way-coaxial-car-speakers-with-polypropylene-cones-pair-black/1807206.p?id=1218293494853&skuId=1807206&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
1,6432002,test-w-mapping-v2,22.773060,Kicker,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]",Blast your favorite music while cruising with these Kicker KS Series 41KSC6934 3-way speakers that feature EVC (Extended Voice Coil) technology for thumping bass response. The calibrated-dome tweeters help you hear every detail.,False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/6432002_sb.jpg,4602,149.99,100 - 200,"Kicker - KS Series 6"" x 9"" 3-Way Car Speakers with Polymer Cones (Pair) - Black",3,Speakers,http://www.bestbuy.com/site/kicker-ks-series-6-x-9-3-way-car-speakers-with-polymer-cones-pair-black/6432002.p?id=1219217424553&skuId=6432002&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
2,8063256,test-w-mapping-v2,22.773060,Pioneer,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 8"" Car Speakers]","Featuring 6"" x 8"" IMPP composite woofer cones and 1-3/16"" PET Piezoelectric tweeters, these Pioneer TS-G6845R car speakers project a wide range of crisp, robust tones, so you can enjoy listening to music while you drive.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 8"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/8063256_sb.jpg,2098,49.99,1 - 50,"Pioneer - 6"" x 8"" 2-Way Car Speakers with IMPP Composite Cones (Pair) - Black",1,Speakers,http://www.bestbuy.com/site/pioneer-6-x-8-2-way-car-speakers-with-impp-composite-cones-pair-black/8063256.p?id=1219309057333&skuId=8063256&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
3,3937008,test-w-mapping-v2,22.190817,Kenwood,"[Car Electronics & GPS, Car Audio, Car Speakers, 6"" x 9"" Car Speakers]","These Kenwood Performance Series KFC-6995PS car speakers feature 6"" x 9"" paper woofer cones, 1/2"" PET-dome and 3/8"" ceramic super tweeters and 13/16"" PET balanced-dome tweeters for vivid sonic output. The acoustic sound harmonizer offers clear audio.",False,Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 6"" x 9"" Car Speakers",https://cdn-demo.algolia.com/bestbuy/3937008_rb.jpg,4707,99.99,50 - 100,"Kenwood - Performance Series 6"" x 9"" 5-Way Car Speakers with Paper Cones (Pair) - Black",6,Speakers,http://www.bestbuy.com/site/kenwood-performance-series-6-x-9-5-way-car-speakers-with-paper-cones-pair-black/3937008.p?id=1219094699970&skuId=3937008&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,_doc
4,7456272,te

Finally it works!

# Let's build a search engine!

In [33]:
def search_engine(phrase):
    resp = query_elasticsearch('test-w-mapping-v2', query_with_weight(phrase))
    
    # Get item from response
    items = [res['_source'] for res in resp['hits']['hits']]
    
    item_template = """
      <tr>
        <th style="width: 100px"><img src="{image}"></th>
        <th>{name}</th>
        <th>{brand}</th>
        <th>{description}</th>
      </tr>
    """

    page = ""
    page += """<table style="width: 70%">"""
    for row in items:
        page += item_template.format(image=row['image'], name=row['product_name'], brand=row['brand'], description=row['description'])
    page += "</table>"

    return page

In [34]:
HTML(search_engine("car audio 6 inches"))

,"Alpine - 6"" x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black",Alpine,"Enjoy your favorite songs on the go with these coaxial 2-way car speakers that feature 65Hz - 18kHz frequency response for accurate sound reproduction. The 3/4"" silk soft-dome tweeter provides crisp, unhindered sound."
,"Kicker - KS Series 6"" x 9"" 3-Way Car Speakers with Polymer Cones (Pair) - Black",Kicker,Blast your favorite music while cruising with these Kicker KS Series 41KSC6934 3-way speakers that feature EVC (Extended Voice Coil) technology for thumping bass response. The calibrated-dome tweeters help you hear every detail.
,"Pioneer - 6"" x 8"" 2-Way Car Speakers with IMPP Composite Cones (Pair) - Black",Pioneer,"Featuring 6"" x 8"" IMPP composite woofer cones and 1-3/16"" PET Piezoelectric tweeters, these Pioneer TS-G6845R car speakers project a wide range of crisp, robust tones, so you can enjoy listening to music while you drive."
,"Kenwood - Performance Series 6"" x 9"" 5-Way Car Speakers with Paper Cones (Pair) - Black",Kenwood,"These Kenwood Performance Series KFC-6995PS car speakers feature 6"" x 9"" paper woofer cones, 1/2"" PET-dome and 3/8"" ceramic super tweeters and 13/16"" PET balanced-dome tweeters for vivid sonic output. The acoustic sound harmonizer offers clear audio."
,"Pioneer - 4"" x 6"" 3-Way Car Speakers with Multilayer Mica Matrix Cones (Pair) - Black",Pioneer,"Enhance your vehicle's audio system with these Pioneer TS-A4675R 3-way speakers that each feature a 4"" x 6"" multilayer mica matrix woofer cone and a 3/8"" PET tweeter for rich sound. The heat-resistant voice coil wires ensure optimal input power handling."
,"Pioneer - 6"" x 9"" 2-Way Car Speakers with IMPP Composite Cones (Pair) - Black",Pioneer,"Immerse yourself in vivid audio with these Pioneer TS-G6945R 2-way car speakers, which feature 6"" x 9"" IMPP composite woofer cones to deliver deep, powerful bass. The rubber-coated cloth surround helps ensure robust sound."
,"Alpine - 6-1/2"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black",Alpine,"Listen to your favorite tunes on the go with these 2-way coaxial speakers that feature 240 watts of peak power for impressive sound quality and a 3/4"" silk soft-dome tweeter for clean audio."
,"Alpine - 5"" x 7"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black",Alpine,"Enjoy your music on the go with these 2-way coaxial speakers that deliver up to 230W of peak power for enhanced sound. A 1"" swivel soft-dome tweeter provides crisp, focused highs."
,Alpine - CD - Built-In Bluetooth - Apple® iPod®- and Satellite Radio-Ready In-Dash Receiver - Black,Alpine,"Enjoy your favorite tunes with this Alpine CDE-153BT CD in-dash receiver, which features a front auxiliary input and a front USB port for easy connection of an Apple iPod or iPhone®. Advanced Bluetooth technology enables easy audio streaming."
,"Alpine - 12"" Dual-Voice-Coil 4-Ohm Subwoofer - Black",Alpine,"Listen to music in your vehicle with rich, clear sound with this Alpine SWS-12D4 12"" dual-voice-coil 4-ohm subwoofer that features improved thermal capacity and mechanical strength for reliable performance."
